run this first to see the output prediction based on pre-trained data.

Note: Choose from {'A': 'a', 'B': 'b', 'C': 'c', 'D':'d', 'E':'e', 'F':'f', 'G':'g', 'H':'h', 'K':'k', 'L':'l', 'Y':'y',  'V':'v'}

Code can be altered to do prediction for remaining alphabets also. I have taken into account only 13 alphabets.

alphabets -  J and Z are out of scope.


In [3]:
import sys, os
import operator
import numpy as np
import cv2
from keras.models import model_from_json
from matplotlib import pyplot as plt

#get from saved model while training
json_file = open("saved_model.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)

# load weights into new model
loaded_model.load_weights("saved_model.h5")
print("Model loaded from trained data")

real_time_capture = cv2.VideoCapture(0)

#---------------------------self begin-------------------------------------------------
#alphabets = {'A': 'a', 'B': 'b', 'C': 'c', 'D':'d', 'E':'e','F':'f','G':'g','H':'h','K':'k','L':'l','Y':'y', 'V':'v'}



while True:
    _, frame = real_time_capture.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)

    image = cv2.imread('amer_sign2.png')
    cv2.imshow("sign Language", image)
#---------------------------self end-------------------------------------------------
    
#---------------------------Source code begin-------------------------------------
    #region of interest coordinates
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    #create ROI rectangle
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1+50:y2+50, x1:x2]
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    
    #----------------------self modified code------------------------------
    
    ASL_predit = {'A': result[0][0], 
                  'B': result[0][1], 
                  'C': result[0][2],
                  'D': result[0][3],
                  'E': result[0][4], 
                  'F': result[0][5], 
                  'G': result[0][6],
                  'H': result[0][7],
                  'K': result[0][8], 
                  'L': result[0][9], 
                  'Y': result[0][11],
                  'V': result[0][10]
                 }
    # perform sorting depending on best predited output
    ASL_predit = sorted(ASL_predit.items(), key=operator.itemgetter(1), reverse=True)
    #view the output
    cv2.putText(frame, ASL_predit[0][0], (10, 200), cv2.FONT_HERSHEY_TRIPLEX, 7, (0,0,255), 2)    
    cv2.imshow("Frame", frame)
    
    
    Key_int = cv2.waitKey(10)
    if Key_int & 0xFF == 27: # esc key
        break
        
real_time_capture.release()
cv2.destroyAllWindows()

Model loaded from trained data
